In [ ]:
# Carga de librerías
import numpy as np
import tensorflow as tf
import PIL
import PIL.Image
from PIL import Image
import pathlib
import scipy as sp
import seaborn as sns
from itertools import product
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import time
import torch
import scipy.spatial
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets

In [ ]:
import pickle
import sys
from pathlib import Path

# Añadir el directorio raíz del proyecto a sys.path
module_path = Path('/Users/Cesar/Desktop/Proyecto-CSG/').resolve()
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Añadir el directorio que contiene 'new_spectral_metric' a sys.path
new_spectral_metric_path = module_path / '/Users/Cesar/Desktop/Proyecto-CSG/pytorch_spectral_metric'
if new_spectral_metric_path not in sys.path:
    sys.path.append(str(new_spectral_metric_path))

from numpy.linalg import LinAlgError
from scipy.sparse.csgraph import laplacian
from pytorch_spectral_metric.estimator_pytorch import CumulativeGradientEstimator
from pytorch_spectral_metric.visualize_pytorch import make_graph
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D, Lambda
from tensorflow.keras import backend as K
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.collections import LineCollection
from matplotlib.font_manager import FontProperties
from pathlib import Path

In [ ]:
print("GPU disponible:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name() if torch.cuda.is_available() else "N/A")


In [ ]:
# Definir directorios
directorio_dataset = '../Datasets/CIFAR-10 dos clases'
imagenes_originales = '../Datasets/CIFAR-10 dos clases/cat'
imagenes_corruptas = '../Datasets/CIFAR-10 dos clases/cat_c_defocus_blur'

directorio_dataset = pathlib.Path(directorio_dataset)
imagenes_originales = pathlib.Path(imagenes_originales)
imagenes_corruptas = pathlib.Path(imagenes_corruptas)



# Dimensiones de imagen y tamaño de batch
img_height = 32
img_width = 32
#batch_size = len(list(data_dir.glob('*/*.png'))) # leer todas las imágenes al tiempo


# Transformaciones para preprocesamiento de datos
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),  # Ajustar tamaño a 32x32
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])

class CustomImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = pathlib.Path(directory)
        self.transform = transform
        self.images = list(self.directory.glob('*.png'))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image



# Crear datasets personalizados
dataset_completo = datasets.ImageFolder(root=directorio_dataset, transform=transform)
dataset_originales = CustomImageDataset(imagenes_originales, transform=transform)
dataset_corruptas = CustomImageDataset(imagenes_corruptas, transform=transform)

# Calcular totales
total_dataset = len(dataset_originales) + len(dataset_corruptas)
total_originales = len(dataset_originales)
total_corruptas = len(dataset_corruptas)

batch_size = total_dataset



# Crear el DataLoader para iterar sobre los datos en lotes
# DataLoader para cargar los datasets
dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)
dataloader_originales = DataLoader(dataset_originales, batch_size=batch_size, shuffle=False)
dataloader_corruptas = DataLoader(dataset_corruptas, batch_size=batch_size, shuffle=False)


print(f"Total imágenes dataset: {total_dataset}")
print(f"Total imágenes originales: {total_originales}")
print(f"Total imágenes corruptas: {total_corruptas}")

In [ ]:
# Acceder a las clases

class_names = dataset_completo.classes
num_classes = len(class_names)

print(f"Número de clases: {num_classes}")
print(f"Los nombres de las clases son: {class_names}")

In [ ]:
# Iterar sobre los datos para verificar las etiquetas
for images, labels in dataloader_completo:
    labels = labels.numpy() if hasattr(labels, 'numpy') else labels
    for label in labels:
        assert label in [0, 1], f"Etiqueta inesperada {label} en la imagen."

print("Verificación de etiquetas exitosa.")

In [ ]:
# Iterar sobre los datos para obtener el primer lote
for images, labels in dataloader_completo:
    numpy_all_images = images.numpy()  # Convertir el tensor de imágenes a numpy
    numpy_all_labels = labels.numpy()  # Convertir el tensor de etiquetas a numpy
    
    # Escalar las imágenes al rango [0, 1]
    numpy_all_images /= 255.0
    
    print(numpy_all_images.shape)
    break  # Solo tomamos el primer lote para imprimir las dimensiones